# Greedy coloring and the coloring number

### The Coloring Number of a Graph

The **coloring number** of a graph $G$, denoted as $Col(G)$, is a parameter that provides an **upper bound** on the minimum number of colors required to properly color the graph. It is defined based on a **well-ordering** of the vertices of G.

---

It is enough to apply such an order of deletion of vertices $v_n, v_{n-1}, \ldots, v_1$, that vertex $v_i$ is the vertex with the smallest degree in $$G[V(G) \setminus \{v_n, v_{n-1}, \ldots, v_{i+1}\}]$$ Then $v_1, v_2, \ldots, v_n$ is a good colouring order.

In [11]:
graph = {
    0: [1, 2, 5],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 5],
    3: [1, 2, 4, 6],
    4: [1, 3, 6, 7],
    5: [0, 2, 7, 8],
    6: [3, 4, 7, 9],
    7: [4, 5, 6, 9],
    8: [5, 9],
    9: [6, 7, 8]
}

positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 50),
    5: (100, 200),
    6: (400, 150),
    7: (500, 100),
    8: (50, 250),
    9: (550, 200)
}

In [56]:
graph = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 3],
    3: [1, 2, 4],
    4: [3]
}

positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 100)
}

In [52]:
graph = {
    0: [1, 2, 3],   # Degree = 3
    1: [0, 2],      # Degree = 2
    2: [0, 1],      # Degree = 2
    3: [0, 4],      # Degree = 2
    4: [3]          # Degree = 1
}

positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 100)
}

In [ ]:
graph = {
    0: [1, 2, 3],
    1: [0, 2],
    2: [0, 1, 3, 5],
    3: [0, 2, 4],
    4: [3, 5],
    5: [2, 4]
}

positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 100),
    5: (5)
}

In [66]:
graph = {
    0: [1, 3, 4, 9, 10],
    1: [0, 2],
    2: [1, 3],
    3: [0, 2],
    4: [0, 5, 7],
    5: [4, 6],
    6: [5, 7, 8],
    7: [4, 6],
    8: [6],
    9: [0],
    10: [0]
}

positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 50),
    5: (100, 200),
    6: (400, 150),
    7: (500, 100),
    8: (50, 250),
    9: (550, 200),
    10: (600, 200)
}

In [70]:
import pygame
import time

def greedy_coloring(graph, vertex_order):
    node_colors = {}
    steps = []

    for node in vertex_order:
        neighbor_colors = {node_colors[neighbor] for neighbor in graph[node] if neighbor in node_colors}
        color = 0
        while color in neighbor_colors:
            color += 1
        node_colors[node] = color

        steps.append((node, color, neighbor_colors))

    num_colors_used = max(node_colors.values()) + 1  # Chromatic number
    return node_colors, steps, num_colors_used

# Choose the best vertex ordering using the smallest last vertex ordering (vertices with the smallest degrees go last)
def smallest_last_ordering(graph):
    ordering = []
    remaining = set(graph.keys())

    i = 1
    while remaining:
        min_node = min(remaining, key=lambda node: len([n for n in graph[node] if n in remaining]))
        print(f"Iteration {i}: Remaining nodes: {remaining}")
        print(f"  Selected node: {min_node} (degree {len([n for n in graph[min_node] if n in remaining])})")
        ordering.append(min_node)
        remaining.remove(min_node)
        i += 1

    ordering.reverse()
    print(f"Final smallest-last ordering: {ordering}")
    return ordering

# Choose the best vertex ordering using the largest-first vertex ordering (vertices with the largest degrees go first)
def largest_first_ordering(graph):
    ordering = []
    remaining = set(graph.keys())

    i = 1
    while remaining:
        print(f"Iteration {i}: Remaining nodes: {remaining}")
        max_node = max(remaining, key=lambda node: len([n for n in graph[node] if n in remaining]))
        print(f"  Selected node: {max_node} (degree {len(graph[max_node])})")
        ordering.append(max_node)
        remaining.remove(max_node)
        i += 1

    print(f"Final largest-first ordering: {ordering}")
    return ordering

# Pygame setup for visualization
def visualize_coloring(graph, positions, steps, vertex_order, num_colors_used):
    pygame.init()
    screen = pygame.display.set_mode((600, 400))
    pygame.display.set_caption("Interactive Greedy Graph Coloring with Vertex Ordering")

    palette = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
        (255, 0, 255), (0, 255, 255), (128, 128, 128), (255, 165, 0)
    ]
    white = (255, 255, 255)
    black = (0, 0, 0)
    gray = (200, 200, 200)

    font = pygame.font.Font(None, 24)

    def draw_graph(node_colors, current_node=None, neighbor_colors=None, coloring_number=None):
        screen.fill(white)

        for node, neighbors in graph.items():
            for neighbor in neighbors:
                pygame.draw.line(screen, black, positions[node], positions[neighbor], 1)

        for node, (x, y) in positions.items():
            color = palette[node_colors.get(node, -1)] if node in node_colors else gray
            pygame.draw.circle(screen, color, (x, y), 20)
            pygame.draw.circle(screen, black, (x, y), 20, 1)
            label = font.render(str(node), True, black)
            screen.blit(label, (x - 10, y - 10))

            if node == current_node:
                pygame.draw.circle(screen, (0, 255, 255), (x, y), 25, 2)

            if neighbor_colors and node in graph[current_node]:
                neighbor_color_text = font.render(f"Color: {node_colors.get(node, 'None')}", True, black)
                screen.blit(neighbor_color_text, (x - 30, y + 30))

        if coloring_number is not None:
            coloring_text = font.render(f"Number of Colors Used: {coloring_number}", True, black)
            screen.blit(coloring_text, (20, 20))

        pygame.display.flip()

    # Pygame animation loop
    node_colors = {}
    running = True
    for i, (node, color, neighbor_colors) in enumerate(steps):
        if not running:
            break

        draw_graph(node_colors, current_node=node, neighbor_colors=neighbor_colors)

        available_colors = font.render(f"Step {i + 1}: Node {node}, Color {color}", True, black)
        sorted_sequence = font.render(f"Vertex Order: {vertex_order}", True, black)
        screen.blit(available_colors, (20, 350))
        screen.blit(sorted_sequence, (20, 320))
        pygame.display.flip()

        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    waiting = False
                if event.type == pygame.KEYDOWN:
                    waiting = False

        node_colors[node] = color
        draw_graph(node_colors)

    draw_graph(node_colors, coloring_number=num_colors_used)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()

coloring_number = max_degree = max((len(graph[v]) for v in graph))
coloring_number = max_degree + 1
print(f"The coloring number is {coloring_number}")

The coloring number is 6


## Smallest-last ordering

In [71]:
vertex_order = smallest_last_ordering(graph)
colors, steps, num_colors_used = greedy_coloring(graph, vertex_order)
visualize_coloring(graph, positions, steps, vertex_order, num_colors_used)

Iteration 1: Remaining nodes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
  Selected node: 8 (degree 1)
Iteration 2: Remaining nodes: {0, 1, 2, 3, 4, 5, 6, 7, 9, 10}
  Selected node: 9 (degree 1)
Iteration 3: Remaining nodes: {0, 1, 2, 3, 4, 5, 6, 7, 10}
  Selected node: 10 (degree 1)
Iteration 4: Remaining nodes: {0, 1, 2, 3, 4, 5, 6, 7}
  Selected node: 1 (degree 2)
Iteration 5: Remaining nodes: {0, 2, 3, 4, 5, 6, 7}
  Selected node: 2 (degree 1)
Iteration 6: Remaining nodes: {0, 3, 4, 5, 6, 7}
  Selected node: 3 (degree 1)
Iteration 7: Remaining nodes: {0, 4, 5, 6, 7}
  Selected node: 0 (degree 1)
Iteration 8: Remaining nodes: {4, 5, 6, 7}
  Selected node: 4 (degree 2)
Iteration 9: Remaining nodes: {5, 6, 7}
  Selected node: 5 (degree 1)
Iteration 10: Remaining nodes: {6, 7}
  Selected node: 6 (degree 1)
Iteration 11: Remaining nodes: {7}
  Selected node: 7 (degree 0)
Final smallest-last ordering: [7, 6, 5, 4, 0, 3, 2, 1, 10, 9, 8]


## Largest-first ordering

In [72]:
vertex_order = largest_first_ordering(graph)
colors, steps, num_colors_used = greedy_coloring(graph, vertex_order)
visualize_coloring(graph, positions, steps, vertex_order, num_colors_used)

Iteration 1: Remaining nodes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
  Selected node: 0 (degree 5)
Iteration 2: Remaining nodes: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
  Selected node: 6 (degree 3)
Iteration 3: Remaining nodes: {1, 2, 3, 4, 5, 7, 8, 9, 10}
  Selected node: 2 (degree 2)
Iteration 4: Remaining nodes: {1, 3, 4, 5, 7, 8, 9, 10}
  Selected node: 4 (degree 3)
Iteration 5: Remaining nodes: {1, 3, 5, 7, 8, 9, 10}
  Selected node: 1 (degree 2)
Iteration 6: Remaining nodes: {3, 5, 7, 8, 9, 10}
  Selected node: 3 (degree 2)
Iteration 7: Remaining nodes: {5, 7, 8, 9, 10}
  Selected node: 5 (degree 2)
Iteration 8: Remaining nodes: {7, 8, 9, 10}
  Selected node: 7 (degree 2)
Iteration 9: Remaining nodes: {8, 9, 10}
  Selected node: 8 (degree 1)
Iteration 10: Remaining nodes: {9, 10}
  Selected node: 9 (degree 1)
Iteration 11: Remaining nodes: {10}
  Selected node: 10 (degree 1)
Final largest-first ordering: [0, 6, 2, 4, 1, 3, 5, 7, 8, 9, 10]
